In [23]:
import pandas as pd
import numpy as np

In [3]:
movies = pd.read_csv('../data/imdb-processed.csv')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4329 entries, 0 to 4328
Data columns (total 70 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   4329 non-null   object 
 1   title                4329 non-null   object 
 2   genres               4329 non-null   object 
 3   weighted_rating      4329 non-null   float64
 4   release_year         4329 non-null   int64  
 5   rating_1             4329 non-null   int64  
 6   rating_2             4329 non-null   int64  
 7   rating_3             4329 non-null   int64  
 8   rating_4             4329 non-null   int64  
 9   rating_5             4329 non-null   int64  
 10  rating_6             4329 non-null   int64  
 11  rating_7             4329 non-null   int64  
 12  rating_8             4329 non-null   int64  
 13  rating_9             4329 non-null   int64  
 14  rating_10            4329 non-null   int64  
 15  country_0            4329 non-null   o

In [4]:
movies['total_votes'] = movies.loc[:, 'rating_1':'rating_10'].sum(axis='columns')

In [5]:
def calc_polarization(row):
    total_score = 0
    for rating_1 in range(1, 11):
        for rating_2 in range(rating_1 + 1, 11):
            total_score += row[f'rating_{rating_1}'] / row['total_votes'] * row[f'rating_{rating_2}'] / row['total_votes'] * (rating_2 - rating_1)

    return total_score

In [6]:
movies['polarization_score'] = movies.apply(calc_polarization, axis='columns')

In [9]:
movies.sort_values('polarization_score', ascending=False).head(25)

,id,title,genres,weighted_rating,release_year,rating_1,rating_2,rating_3,rating_4,rating_5,...,country_4_rating_3,country_4_rating_4,country_4_rating_5,country_4_rating_6,country_4_rating_7,country_4_rating_8,country_4_rating_9,country_4_rating_10,total_votes,polarization_score
3314,tt4776998,The Promise,"Action,Adventure,Drama",6.1,2017,71664,785,464,775,1792,...,0,3,3,19,33,24,53,4487,179364,2.203718
4326,tt7668842,Enes Batur: Imagination or Reality?,Comedy,2.0,2018,21542,748,526,347,471,...,0,1,0,0,0,0,0,176,56291,2.186839
4283,tt6277462,Brahmastra Part One: Shiva,"Action,Adventure,Fantasy",5.6,2022,42059,3965,2358,2499,3614,...,39,40,58,69,77,71,70,359,113603,2.150143
3628,tt26932223,Bhool Bhulaiyaa 3,"Comedy,Fantasy,Horror",4.7,2024,22601,5180,4905,3708,1539,...,27,32,42,38,30,20,8,77,76926,2.138212
4305,tt12915716,Adipurush,"Action,Drama",2.7,2023,51121,3081,1575,1054,1077,...,16,5,5,2,3,5,2,51,135200,2.120026
4172,tt10028196,Laal Singh Chaddha,"Comedy,Drama,Romance",5.6,2022,97775,4265,1436,973,1495,...,41,53,92,130,172,180,142,900,179265,2.107484
4106,tt10083340,Gangubai Kathiawadi,"Biography,Crime,Drama",7.8,2022,16466,1161,763,856,1517,...,12,15,37,79,160,186,103,178,63758,1.976030
4055,tt12844910,Pathaan,"Action,Adventure,Thriller",5.8,2023,36924,3847,3083,3522,5861,...,73,83,133,164,151,96,385,1235,160133,1.971061
1626,tt0368226,The Room,Drama,3.6,2003,32820,12662,8910,5200,4666,...,303,139,141,104,103,87,50,596,97179,1.956152
4324,tt11045422,Toofaan,"Drama,Sport",6.1,2021,12899,730,522,603,1027,...,7,12,22,17,29,22,14,47,52214,1.933142


In [11]:
def calc_country_rating(country_idx, row):
    total_rating = 0
    total_votes = 0
    for rating in range(1, 11):
        total_rating += rating * row[f'country_{country_idx}_rating_{rating}']
        total_votes += row[f'country_{country_idx}_rating_{rating}']
    return total_rating / total_votes

In [12]:
for i in range(5):
    movies[f'country_{i}_rating'] = movies.apply(lambda row: calc_country_rating(i, row), axis='columns')

In [30]:
def calc_ptp(row):
    ptp = np.ptp(row.loc['country_0_rating':'country_4_rating'])
    
    return ptp

In [31]:
movies['country_ptp'] = movies.apply(calc_ptp, axis='columns')

In [32]:
movies.sort_values('country_ptp', ascending=False).head(25)

,id,title,genres,weighted_rating,release_year,rating_1,rating_2,rating_3,rating_4,rating_5,...,country_4_rating_9,country_4_rating_10,total_votes,polarization_score,country_0_rating,country_1_rating,country_2_rating,country_3_rating,country_4_rating,country_ptp
3314,tt4776998,The Promise,"Action,Adventure,Drama",6.1,2017,71664,785,464,775,1792,...,53,4487,179364,2.203718,8.853424,1.372856,9.621135,7.973952,9.920069,8.547214
4299,tt11555492,Farha,Drama,8.0,2022,9534,177,97,127,268,...,110,4083,90080,0.981824,9.418896,2.721110,9.675454,9.617452,9.901685,7.180575
4326,tt7668842,Enes Batur: Imagination or Reality?,Comedy,2.0,2018,21542,748,526,347,471,...,0,176,56291,2.186839,5.369399,3.201413,7.219048,3.030189,9.915730,6.885542
4322,tt10554232,Dara of Jasenovac,"Drama,War",8.1,2021,3818,262,176,200,350,...,12,449,82083,0.560421,9.872591,9.197059,9.094617,9.785560,3.023002,6.849588
4314,tt27719898,Kaiva,"Action,Drama,Romance",8.5,2023,7908,1162,16,28,37,...,0,1,68958,1.159006,8.632270,5.288793,7.434343,2.772727,2.846154,5.859543
4271,tt15979666,The Archies,"Comedy,Drama,Musical",4.2,2023,7096,918,579,601,754,...,7,36,53125,1.472463,8.523331,3.764650,3.804031,4.066975,3.327014,5.196317
2683,tt0077928,Midnight Express,"Biography,Crime,Drama",7.5,1978,5628,684,675,1175,2725,...,500,280,90637,1.065465,7.436350,7.554844,3.224610,8.212210,7.648483,4.987600
4272,tt9637132,Samrat Prithviraj,"Action,Drama,History",5.6,2022,7574,964,586,637,927,...,10,53,53725,1.497891,8.384095,5.233802,5.597938,3.492119,4.573034,4.891976
7,tt20221436,Emilia Pérez,"Comedy,Crime,Drama",5.5,2024,24930,3995,3366,3753,5153,...,320,197,73792,1.737361,4.775475,2.465527,1.949050,5.211796,6.801721,4.852670
4328,tt10280276,Coolie No. 1,"Action,Comedy,Romance",4.2,2020,46055,1455,760,576,525,...,1,31,74467,1.910510,3.133739,6.429201,1.851842,4.394161,1.663772,4.765429
